Load titanic data set from sns

check nulls, and impute them

check outleirs and trim/ cap

do feature selection using RFE

import SVC and train and test the model


In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [27]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [28]:
titanic.drop(columns=['who', 'adult_male', 'embarked', 'alive', 'alone', 'class', 'deck'], axis=1, inplace=True)

In [29]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town
0,0,3,male,22.0,1,0,7.2500,Southampton
1,1,1,female,38.0,1,0,71.2833,Cherbourg
2,1,3,female,26.0,0,0,7.9250,Southampton
3,1,1,female,35.0,1,0,53.1000,Southampton
4,0,3,male,35.0,0,0,8.0500,Southampton


In [30]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embark_town      2
dtype: int64

In [31]:
titanic['embark_town'] = titanic['embark_town'].fillna(titanic['embark_town'].mode()[0])
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embark_town      0
dtype: int64

In [32]:
px.box(titanic['age'])

In [33]:
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embark_town    0
dtype: int64

In [34]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder().fit_transform(titanic)

In [35]:
type(ohe)

scipy.sparse._csr.csr_matrix

In [36]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town
0,0,3,male,22.0,1,0,7.2500,Southampton
1,1,1,female,38.0,1,0,71.2833,Cherbourg
2,1,3,female,26.0,0,0,7.9250,Southampton
3,1,1,female,35.0,1,0,53.1000,Southampton
4,0,3,male,35.0,0,0,8.0500,Southampton


In [37]:
titanic = pd.get_dummies(data = titanic, columns=['sex', 'embark_town'], drop_first=True, dtype=int)
titanic.head()

,survived,pclass,age,sibsp,parch,fare,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,1


In [38]:
X= titanic.drop(columns='survived', axis=1)
y=titanic['survived']

In [39]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=5, step=1).fit(X,y)

               


c:\Users\desut\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [40]:
selector.ranking_

array([1, 3, 1, 2, 4, 1, 1, 1])

In [41]:
X.columns

Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male',
       'embark_town_Queenstown', 'embark_town_Southampton'],
      dtype='object')

In [43]:
X_new = X[['pclass', 'sibsp', 'sex_male',
       'embark_town_Queenstown', 'embark_town_Southampton']]

In [45]:
X_new.head()

,pclass,sibsp,sex_male,embark_town_Queenstown,embark_town_Southampton
0,3,1,1,0,1
1,1,1,0,0,0
2,3,0,0,0,1
3,1,1,0,0,1
4,3,0,1,0,1


In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=0)

In [48]:
from sklearn.svm import SVC
svc = SVC().fit(X_train, y_train)

In [49]:
ypred = svc.predict(X_test)

In [50]:
from sklearn import metrics
metrics.accuracy_score(y_test, ypred)

0.7932960893854749